In [15]:
import random
import pickle
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [10]:
text_df = pd.read_csv("fake_or_real_news.csv")

In [11]:
text = list(text_df.text.values)
joined_text = " ".join(text)

In [12]:
partical_text = joined_text[:10000]

In [16]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partical_text.lower())

In [18]:
unique_tokens = np.unique(tokens)
unique_tokens_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [22]:
n_words = 10
input_words = []
next_words = []

for i in range(len(tokens) - n_words):
  input_words.append(tokens[i:i + n_words])
  next_words.append(tokens[i + n_words])

In [27]:
x = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [30]:
for i, words in enumerate(input_words):
  for j, word in enumerate(words):
    x[i, j, unique_tokens_index[word]] = 1
  y[i, unique_tokens_index[next_words[i]]] = 1

In [35]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [40]:
model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])
model.fit(x, y, batch_size=128, epochs=30, shuffle=True)

Epoch 1/30
14/14 [==============================] - 8s 102ms/step - loss: 6.1995 - accuracy: 0.0458
Epoch 2/30
14/14 [==============================] - 1s 99ms/step - loss: 5.8468 - accuracy: 0.0618
Epoch 3/30
14/14 [==============================] - 1s 100ms/step - loss: 5.7964 - accuracy: 0.0618
Epoch 4/30
14/14 [==============================] - 1s 102ms/step - loss: 5.7858 - accuracy: 0.0618
Epoch 5/30
14/14 [==============================] - 1s 102ms/step - loss: 5.7516 - accuracy: 0.0618
Epoch 6/30
14/14 [==============================] - 2s 170ms/step - loss: 5.7182 - accuracy: 0.0618
Epoch 7/30
14/14 [==============================] - 2s 117ms/step - loss: 5.6788 - accuracy: 0.0595
Epoch 8/30
14/14 [==============================] - 1s 99ms/step - loss: 5.5865 - accuracy: 0.0618
Epoch 9/30
14/14 [==============================] - 1s 102ms/step - loss: 5.4954 - accuracy: 0.0646
Epoch 10/30
14/14 [==============================] - 1s 105ms/step - loss: 5.3642 - accuracy: 0.0635
E

In [41]:
model.save("mymodel.h5")

In [42]:
model = load_model("mymodel.h5")

In [48]:
def predict_next_word(input_text, n_best):
  input_text = input_text.lower()
  x = np.zeros((1, n_words, len(unique_tokens)))
  for i, word in enumerate(input_text.split()):
    x[0, i, unique_tokens_index[word]] = 1
  

  predictions = model.predict(x)[0]
  return np.argpartition(predictions, n_best)[n_best:]

In [49]:
possible = predict_next_word("He will have to look into his thing as he", 5)

1/1 [==============================] - 0s 48ms/step


In [51]:
print([unique_tokens[idx] for idx in possible])

['don', 'right', 'results', 'plan', 'well', 'television', 'damaging', 'her', 'left', 'tenth', 'outcome', 'corruption', 'islam', 'got', 'often', 'retired', 'coy', 'crime', 'away', 'react', 'bigger', 'may', 'fundamental', 're', 'up', 'quickly', 'illegality', 'whatever', 'amendment', 'impropriety', 'but', 'cable', 'off', 'associates', 'any', 'and', 'years', 'you', 'bribery', 'no', 'destroy', 'clintonworld', 'possible', 'especially', 'come', 'illegal', 'entire', 'victory', 'credibility', 'weathered', 'other', 'them', 'stay', 'at', 'then', 'assaults', 'that', 'they', 'idea', 'pieces', 'back', 'playing', 'thing', 'those', 'hubris', 'bathroom', 'edgar', 'around', 'however', 'how', 'before', 'behavior', 't', 'establishment', 'belief', 'believes', 'longer', 'home', 'substance', 'thought', 'word', 'wing', 'threat', 'throughout', 'have', 'more', 'place', 'breeze', 'most', 'train', 'bring', 'whole', 'buried', 'truly', 'writer', 'try', 'even', 'james', 'hacks', 'spectrum', 'greenfield', 'york', 'sm